In [ ]:
# Restart Kernel
!pip install -qq -U bitsandbytes

In [1]:
import pandas as pd
import numpy as np

import shutil
from pathlib import Path
import os
import random
import re
import json

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
from PIL import Image
from tqdm import tqdm
from datasets import load_dataset

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

from peft import PeftModel
from transformers import (AutoModelForCausalLM, Blip2Processor, Blip2Model, BlipImageProcessor, AutoTokenizer, BitsAndBytesConfig,
    Trainer, TrainingArguments, default_data_collator, TrainerCallback)

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

from datasets import load_dataset



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

save_path = '/kaggle/working/'

2025-07-04 18:14:28.761485: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751652868.981667      88 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751652869.045839      88 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cuda


In [2]:
class BLIP2ForPhi(nn.Module):
    def __init__(self, vision_model, q_former, phi_model, query_tokens):
        super().__init__()
        self.vision_model = vision_model
        self.q_former = q_former
        self.projection = nn.Linear(q_former.config.hidden_size, phi_model.config.hidden_size)
        self.phi_model = phi_model
        self.query_tokens = query_tokens

    
    def forward(self, pixel_values, input_ids, attention_mask, labels=None):
        image_embeds = self.vision_model(pixel_values).last_hidden_state

        batch_size = image_embeds.shape[0]
        qformer_query_embeds = self.query_tokens.expand(batch_size, -1, -1)

        
        image_attention_mask = torch.ones(image_embeds.size()[:-1], dtype=torch.long, device=image_embeds.device)
        query_outputs = self.q_former(
            query_embeds=qformer_query_embeds,
            encoder_hidden_states=image_embeds,
            encoder_attention_mask=image_attention_mask
        )[0]

        projected_query = self.projection(query_outputs)

        text_embeds = self.phi_model.get_input_embeddings()(input_ids)
        inputs_embeds = torch.cat([projected_query, text_embeds], dim=1)
        
        # 입력 ID의 attention_mask와 쿼리의 attention_mask를 결합
        query_attention_mask = torch.ones(projected_query.size()[:-1], dtype=torch.long, device=projected_query.device)
        combined_attention_mask = torch.cat([query_attention_mask, attention_mask], dim=1)

        
        outputs = self.phi_model(
            inputs_embeds=inputs_embeds,
            attention_mask=combined_attention_mask,
            labels=labels, 
        )
        return outputs
        

    def generate(self, pixel_values, input_ids, attention_mask, **generate_kwargs):
        image_embeds = self.vision_model(pixel_values).last_hidden_state
        batch_size = image_embeds.shape[0]
        qformer_query_embeds = self.query_tokens.expand(batch_size, -1, -1)
        image_attention_mask = torch.ones(image_embeds.size()[:-1], dtype=torch.long, device=image_embeds.device)
        
        query_outputs = self.q_former(
            query_embeds=qformer_query_embeds,
            encoder_hidden_states=image_embeds,
            encoder_attention_mask=image_attention_mask
        )[0]
        
        projected_query = self.projection(query_outputs) # shape: [B, 32, D_phi]

        # 2. 텍스트 처리 -> 텍스트 임베딩 생성
        text_embeds = self.phi_model.get_input_embeddings()(input_ids) # shape: [B, S, D_phi]

        # 3. 이미지 임베딩과 텍스트 임베딩 결합
        inputs_embeds = torch.cat([projected_query, text_embeds], dim=1)
        
        # 4. 어텐션 마스크도 결합
        query_attention_mask = torch.ones(projected_query.size()[:-1], dtype=torch.long, device=projected_query.device)
        combined_attention_mask = torch.cat([query_attention_mask, attention_mask], dim=1)

        # 5. 결합된 임베딩으로 LLM의 generate 함수 호출
        outputs = self.phi_model.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=combined_attention_mask,
            **generate_kwargs
        )
        return outputs

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)


image_processor = BlipImageProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
if phi_tokenizer.pad_token is None:
    phi_tokenizer.pad_token = phi_tokenizer.eos_token

blip2_model = Blip2Model.from_pretrained("Salesforce/blip2-opt-2.7b")

vision_model = blip2_model.vision_model
q_former = blip2_model.qformer
query_tokens = blip2_model.query_tokens

phi_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    quantization_config=quantization_config,
    trust_remote_code=True,
)

preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [4]:
def load_checkpoint(model, checkpoint_path: str):
        checkpoint = torch.load(checkpoint_path, map_location=device)
        
        model.load_state_dict(checkpoint['model_state_dict'], strict=False)
        print(f"✅ Checkpoint loaded")

In [11]:
model = BLIP2ForPhi(vision_model, q_former, phi_model, query_tokens)
load_checkpoint(model, '/kaggle/input/blip2_epoch11/pytorch/default/1/epoch_11/checkpoint.pt')
model.to(device)
model.eval();

Freezing vision_model and phi_model parameters...
Training q_former and projection layer...
✅ Checkpoint loaded


### Testset

In [8]:
test = pd.read_csv('/kaggle/input/scpc-testset/test.csv')

In [21]:
def extract_answer_letter(text):
    match = re.search(r"Answer:\s*([A-Da-d])\b", text)
    return match.group(1).upper() if match else "?"

test = pd.read_csv('/kaggle/input/scpc-testset/test.csv')
results = []


for _, row in tqdm(test.iterrows(), total=len(test)):
    image = Image.open(os.path.join('/kaggle/input/scpc-testset', row['img_path'])).convert("RGB")
    choices = [row[c] for c in ['A', 'B', 'C', 'D']]

    prompt = (
        "You are a helpful AI that answers multiple-choice questions based on the given image.\n" +
        f"Select only the single best answer from A, B, C, or D.\n\n"
        f"Respond only with one letter (e.g., 'A').\n\n"
        f"Question: {row['Question']}\n\n" +
        "\n".join([f"{chr(65+i)}. {choice}" for i, choice in enumerate(choices)]) +
        "\n\nAnswer:"
        )

    image_inputs = image_processor(image, return_tensors="pt")
    text_inputs = phi_tokenizer(prompt, return_tensors="pt")
    
    inputs = {
            "pixel_values": image_inputs["pixel_values"].to(device),
            "input_ids": text_inputs["input_ids"].to(device),
            "attention_mask": text_inputs["attention_mask"].to(device)
        }
        
    # 모델의 업그레이드된 generate 함수 호출
    with torch.autocast(device_type="cuda", dtype=torch.float16):
        output_ids = model.generate(
            **inputs, 
            max_new_tokens=5,
            do_sample=False,
            temperature=0.0,
            pad_token_id=phi_tokenizer.eos_token_id
        )  

    decoded = phi_tokenizer.decode(output_ids[0], skip_special_tokens=True).strip()
    results.append(decoded[0] if decoded else '?')

print('Done.')


  0%|          | 0/852 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
  0%|          | 1/852 [00:00<06:26,  2.20it/s]

C


  0%|          | 2/852 [00:00<06:14,  2.27it/s]

C


  0%|          | 3/852 [00:01<06:12,  2.28it/s]

...

B


100%|█████████▉| 851/852 [06:12<00:00,  2.25it/s]

C


100%|██████████| 852/852 [06:13<00:00,  2.28it/s]

B
Done.


In [22]:
submission = pd.read_csv('/kaggle/input/scpc-testset/sample_submission.csv')
submission['answer'] = results
submission.to_csv('./blip2_phi1.5_submit.csv', index=False)
print("✅ Done.")

✅ Done.


In [23]:
test['answer'] = results
test.to_csv('./testwithprediction.csv', index=False)
print("✅ Done.")

✅ Done.


In [25]:
test

,ID,img_path,Question,A,B,C,D,answer
0,TEST_000,./test_input_images/TEST_000.jpg,What types of fruits are visible in the image?,Bananas and grapes placed in baskets,Apples and oranges displayed on the counter,Peaches and plums in a wooden crate,Pears and lemons arranged neatly,C
1,TEST_001,./test_input_images/TEST_001.jpg,What type of cuisine are these dishes most com...,"Chinese, famous for dumplings and noodles.","Mexican, featuring tacos and enchiladas.","Italian, known for pasta dishes and pizzas.","Indian, rich in spices and curries.",C
2,TEST_002,./test_input_images/TEST_002.jpg,What is a common reason people doing this acti...,For attending concerts in open-air settings,For physical exercise and scenic views of the ...,For shopping opportunities in remote locations,For fishing in mountain streams,B
3,TEST_003,./test_input_images/TEST_003.jpg,"What type of clothing are the people wearing, ...","Light clothing, suggesting a warm and sunny cl...","Swimwear, suggesting a tropical and beach-like...","Heavy coats, suggesting a cold and possibly sn...","Raincoats, suggesting rainy and wet weather co...",B
4,TEST_004,./test_input_images/TEST_004.jpg,What might the woman have done just before thi...,Landing from a jump or completing a spin in th...,Practiced her routine several times to perfect...,Chatting with other skaters at the boards near...,Sitting down on the ice to take a break and rest.,B
...,...,...,...,...,...,...,...,...
847,TEST_847,./test_input_images/TEST_847.jpg,What is the significance of the objects in man...,They are seen as symbols of luck and prosperity,They are considered omens of bad weather appro...,They are viewed as signs of danger and caution,They are often associated with patience and cr...,A
848,TEST_848,./test_input_images/TEST_848.jpg,Why might the dog be sitting still in this wea...,It is trying to catch a frisbee thrown in the air,It is chasing another animal across the snowy ...,It is waiting for a command from its owner nearby,It is scared of the snow falling from the sky,C
849,TEST_849,./test_input_images/TEST_849.jpg,What is the woman holding in the image?,A long-handled broom for sweeping floors,A spray bottle filled with cleaning solution,A powerful vacuum cleaner for carpets,A fluffy duster for removing dust,B
850,TEST_850,./test_input_images/TEST_850.jpg,What might have happened earlier in the day be...,A market opened nearby selling fresh produce,A parade took place with colorful floats,The gondolier prepared the boat for the day's ...,"A storm occurred, leaving puddles on the streets",C
